In [1]:
import requests, json
import re
from lxml import etree

url = 'https://music.163.com/discover/artist'
singer_infos = []


# ---------------通过url获取该页面的内容，返回xpath对象
def get_xpath(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    return etree.HTML(response.text)

def get_respanse_text(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    return response.text


# --------------通过get爬取到页面后，我们获取华宇，华宇男等分类
def parse():
    html = get_respanse_text(url)
    partten = "/discover/artist/cat\?id=\d{4}"
    fenlei_url_list = re.findall(partten,html)
    # --------将热门和推荐两栏去掉筛选
    new_list = [i for i in fenlei_url_list if 'id' in i]
    for i in new_list:
        fenlei_url = 'https://music.163.com' + i
        parse_fenlei(fenlei_url)
        # print(fenlei_url)


# -------------通过传入的分类url，获取A,B，C页面内容
def parse_fenlei(url):
    html = get_xpath(url)
    # 获得字母排序，每个字母的链接
    zimu_url_list = html.xpath('//ul[@id="initial-selector"]/li[position()>1]/a/@href')
    for i in zimu_url_list:
        zimu_url = 'https://music.163.com' + i
        parse_singer(zimu_url)


# ---------------------传入获得的字母链接，开始爬取歌手内容
def parse_singer(url):
    html = get_xpath(url)
    item = {}
    singer_names = html.xpath('//ul[@id="m-artist-box"]/li/p/a/text()')
    # --详情页看到页面结构会有两个a标签，所以取第一个
    singer_href = html.xpath('//ul[@id="m-artist-box"]/li/p/a[1]/@href')
    # print(singer_names,singer_href)
    for i, name in enumerate(singer_names):
        item['歌手名'] = name
        item['音乐链接'] = 'https://music.163.com' + singer_href[i].strip()
        # 获取歌手详情页的链接
        url = item['音乐链接'].replace(r'?id', '/desc?id')
        # print(url)
        parse_detail(url, item)

        print(item)


# ---------获取详情页url和存着歌手名字和音乐列表的字典，在字典中添加详情页数据
def parse_detail(url, item):
    html = get_xpath(url)
    desc_list = html.xpath('//div[@class="n-artdesc"]/p/text()')
    item['歌手信息'] = desc_list
    singer_infos.append(item)
    write_singer(item)


# ----------------将数据字典写入歌手文件
def write_singer(item):
    with open('singer.json', 'a+', encoding='utf-8') as file:
        json.dump(item, file)

In [2]:
parse()

{'歌手名': '阿冗', '音乐链接': 'https://music.163.com/artist?id=30116848', '歌手信息': ['歌手阿冗，不要音乐旗下艺人，代表作《与我无关》《你的答案》\n合作请联系：hlh@buyao.com']}
{'歌手名': 'Aioz', '音乐链接': 'https://music.163.com/artist?id=29235210', '歌手信息': ['Aioz 北京科技大学一名普通学生 CUBA一名球员 rapper里最能扣篮的\n合作 remix 私信']}
{'歌手名': '艾辰', '音乐链接': 'https://music.163.com/artist?id=12174057', '歌手信息': ['演唱风格：古风 流行音乐 蓝调 摇滚乐 翻唱等\n微博搜索：艾辰辰cur\nQQ搜索：819798358']}
{'歌手名': '阿虾', '音乐链接': 'https://music.163.com/artist?id=31093891', '歌手信息': ['1111111111111111111111111111111111111111']}
{'歌手名': '阿禹ayy', '音乐链接': 'https://music.163.com/artist?id=33164701', '歌手信息': ['即将十八的小青年，热爱音乐与创作']}
{'歌手名': '艾热 AIR', '音乐链接': 'https://music.163.com/artist?id=1203045', '歌手信息': ['歌手、音乐制作人，擅长说唱而不仅仅止于说唱的新生代唱作人。\n代表作：〈乌云中〉、〈巨人〉、〈星球坠落〉。\n成长、突破、蜕变，不愿被定义，不局限说唱，音乐从不设限。\n业界评价：不可估量的艾热！他带来的是一种对HipHop的最全新的认知，是一种态度。不会步步紧逼的引人夺目，不张扬，不虚饰，不在狭小的生活圈里模仿。用最大的热情与爱意，心有所定，专注做事。']}
{'歌手名': '暗杠', '音乐链接': 'https://music.163.com/artist?id=1198010', '歌手信息': ['暗杠，音乐人，唱作人。自2013年起先后发布《 友爱 》 、 《 唱谣 》 、 《 一念 》 、《

{'歌手名': 'Capper', '音乐链接': 'https://music.163.com/artist?id=12193174', '歌手信息': ["Capper，唱作音乐人、平面设计师、街头艺术家。\nCapper,whose real name is yanzhuo Zhang,was born on June 21,2000 in Xi'an,Shannxi Province,the first ancient capital of China,sa a professional rapper and street graphic designer in the '00's.\n微博：@Capper不戴帽子\nINS：cappermadeinfuture"]}
{'歌手名': '陈小春', '音乐链接': 'https://music.163.com/artist?id=2112', '歌手信息': ['陈小春（Jordan Chan），汉族客家人，1967年7月8日出生于广东省惠州市惠阳区，中国香港男歌手、演员。\n1985年陈小春加入TVB舞蹈艺员训练班，先后担任组合豹小子、风火海的成员。1994年凭借电影《晚九朝五》获得第14届香港电影金像奖最佳男配角奖。1996年起参演系列电影《古惑仔》，受到关注。1998年领衔主演金庸武侠剧《鹿鼎记》。1999年凭借歌曲《神啊救救我》《没那种命》拿下当年的香港叱咤风云奖。2003年发行普通话专辑《算你狠》；同年首度于香港体育馆举行个人演唱会。2004年领衔主演古装轻喜剧《天下第一媒》。2005年领衔主演温情连续剧《我家不打烊》。2007年主演古装动作剧《霍元甲》。2008年发行普通话专辑《独家记忆》。2011年11月29日出演的古装战争动作电影《鸿门宴传奇》上映。2013年主演喜剧电影《好命先生》。2014年，在香港红磡体育馆合作举办岁月友情演唱会。2015年，领衔主演黑色喜剧电影《槑计划》。2016年，参加结构式综艺《来吧，说做就做》。2017年领衔主演警匪电影《黑白迷宫》。', '相依为命、独家记忆、算你狠、没那种命、乱世巨星、离不开你、取消资格、我爱的人、犯贱、献世、战无不胜、抱一抱、叱咤红人、神啊救救我、失恋王', 'TVB劲歌金曲卓越表现大奖金奖', '十大中文金曲卓越表现大奖金奖', '劲爆港台

{'歌手名': '迪克牛仔', '音乐链接': 'https://music.163.com/artist?id=2516', '歌手信息': [' 台湾男歌手迪克牛仔从25岁开始舞台生涯，曾组团演唱，历尽坎坷，直到40岁时机遇来临，迅速成为家喻户晓的当红歌星。迪克牛仔演唱的《一言难尽》、《爱》等歌曲在歌迷中广为传唱，他的成功被演艺界誉为“台湾传奇”。几分沧桑，几分激昂，大气恢弘，极具感染力的声音在诉说生命的意义。尽管老爹演绎的绝大多数歌曲和原唱有几分雷同，但依然凭借真情的自然流露打动着每一个听众的心。老爹以人生道路为底蕴，留给大家的是扑面而来的真挚，是那份在生命中呐喊搏击的激情。', '放手去爱、\xa0有多少爱可以重来、\xa0三万英尺、 风飞沙、\xa0梦醒时分、哭不出来、\xa0光辉岁月、老爹、\xa0酒干倘卖无、\xa0你知道我在等你吗、\xa0傻子的约定等。', '我想你了\xa0', '非常都市\xa0', '有多少爱可以重来', '一言难尽', '迪克牛仔曾历经苦难，干过挑砖的建筑工人，但也练就了强壮的身体。', '几分沧桑，几分激昂，大气恢弘，极具感染力的声音在诉说生命的意义。尽管老爹演绎的绝大多数歌曲和原唱有几分雷同，但依然凭借真情的自然流露打动着每一个听众的心。老爹以人生道路为底蕴，留给大家的是扑面而来的真挚，是那份在生命中呐喊搏击的激情。', '1998年老爹从故乡高雄出发，离乡背井的闯荡出了自己的摇滚传奇，也圆了自己的梦。回顾这10年的演唱生涯，老爹觉得自己就很像随风飞扬的风飞沙，离开自己的故乡，一站又一站的前进，始终没有停下脚步。因此在出道10周年的现在，在老爹心中有了这种的想法和感慨。', '1999年，迪克牛仔发行了第一张的全新国语专辑“忘记我还是忘记他”，搭配着港剧的主题曲“忘记我还是忘记他”以及专辑中的主打歌“三万英尺”都是当时红极一时的歌曲，迪克牛仔所带动的摇滚热潮持续不断的加温。', '而2000年1月发行了“我这个你不爱的人”之后，迪克牛仔就开始了他横跨华语乐坛的摇滚传奇之旅。在台湾、香港、星马、中国大陆各地巡回演唱，摇滚教主的现场功力和舞台魅力风靡了全亚洲的华语乐坛。', '2007年，迪克牛仔终于又推出了这张全新的国语专辑“风飞沙”。这张专辑对于迪克牛仔来说可说是别具意义，因为今年是老爹以迪克牛仔为名字出道的第10年。对

{'歌手名': 'FIVESTAR', '音乐链接': 'https://music.163.com/artist?id=51613721', '歌手信息': ['FIVESTARFIVESTAR']}
{'歌手名': '法老', '音乐链接': 'https://music.163.com/artist?id=865007', '歌手信息': ['法老（1992年10月11日-），出生于浙江省海宁市，中国内地说唱歌手。著名硬核chop说唱歌手，活死人厂牌主理人杀人狂魔鼠尾草，afsc厂牌签约艺人，代表中文说唱第六代。']}
{'歌手名': 'Franky弗兰奇', '音乐链接': 'https://music.163.com/artist?id=12338185', '歌手信息': ['深圳新势界music studio主理人 制作人Fanky 弗兰奇 合作请联系 vx：S769799730']}
{'歌手名': 'FOX胡天渝', '音乐链接': 'https://music.163.com/artist?id=1199064', '歌手信息': ['FOX胡天渝,\n音乐人、唱作歌手，\n生于新疆伊犁伊宁，\n代表作《庆功酒》、《宿命》、《枭雄》。']}
{'歌手名': '方续Cuz', '音乐链接': 'https://music.163.com/artist?id=35726654', '歌手信息': ['新浪微博: 方续Cuz，词曲唱一体机，书荒派关门大弟子，流行,hiphop,音乐风格创作者。']}
{'歌手名': 'Forest龙楠林', '音乐链接': 'https://music.163.com/artist?id=34618850', '歌手信息': ['HIPHOP音乐人 和你一样喜欢音乐\nVB：龙楠林 粉丝Q群：664923953\nV:LNGLNL']}
{'歌手名': '费玉清', '音乐链接': 'https://music.163.com/artist?id=2747', '歌手信息': ['费玉清（Yu-Ching Fei），本名张彦亭，1955年7月17日出生于中国台湾省台北市，祖籍安徽省安庆市桐城市，中国台湾男歌手、主持人。\n1973年参加中视举办的《星对星》歌唱比赛，并获得第四名，从而开始歌唱事业。1977年推出首张专辑

{'歌手名': '高宇Slient', '音乐链接': 'https://music.163.com/artist?id=12172312', '歌手信息': ['WeChat： Rap8000']}
{'歌手名': '顾帅Gs', '音乐链接': 'https://music.163.com/artist?id=12898527', '歌手信息': ['顾帅Gs微博同名，都市99线自由相声说唱演员！']}
{'歌手名': '葛东琪', '音乐链接': 'https://music.163.com/artist?id=12136078', '歌手信息': ['葛东琪，中国内地流行歌手，原创音乐制作人。代表作：《囍（Chinese Wedding）》、《悬溺》、《灯塔》等。']}
{'歌手名': '高旭', '音乐链接': 'https://music.163.com/artist?id=33900743', '歌手信息': ['经纪人V：Dingingingingingng']}
{'歌手名': '孤矢', '音乐链接': 'https://music.163.com/artist?id=1194094', '歌手信息': ['来自上海1SHOT厂牌，夜吟人（孤矢&颠倒）\n每个不同的时期 做着同样喜欢的音乐']}
{'歌手名': 'GAI周延', '音乐链接': 'https://music.163.com/artist?id=1211046', '歌手信息': ['GAI周延（D.M.G）种梦音乐签约艺人。\n新华流说唱领军人物。《我是唱作人》冠军战队，《我们的歌》冠军组合，《中国新说唱2020》明星主理人《2021少年说唱企划》《草莓星球来的人》导师，《披荆斩棘的哥哥》唱演家族成员。代表作：《华夏》《烈火战马》《哪吒》《赤子心》等，作品中的中国古典元素、音律融合川渝方言，加之侠义悠然的中国风、稳健的唱功，让GAI周延成为中文说唱最具特色与实力的音乐人。']}
{'歌手名': '鬼卞', '音乐链接': 'https://music.163.com/artist?id=1008034', '歌手信息': ['故弄玄虚, 贩卖思想.']}
{'歌手名': 'h3R3', '音乐链接': 'https://music.163.com/artist?id=126314

{'歌手名': '胡彦斌', '音乐链接': 'https://music.163.com/artist?id=3066', '歌手信息': ['胡彦斌（Tiger Hu），1983年7月4日出生于上海市，祖籍浙江省宁波市，中国内地男歌手、作曲人、音乐制作人，牛班音乐学校创始人。\n1999年参加“上海亚洲音乐节新人歌手大赛”出道。2001年推出个人创作单曲《和尚》获得广泛关注。2002年发行首张个人专辑《文武双全》。2004年在台湾HITO颁奖礼中凭借创作歌曲《进行式》获得最受欢迎对唱金曲奖。2008年在台湾举行“胡彦斌2008男人歌演唱会”，演唱会上座率达9.5成。2014年成立个人工作室“上海太歌文化传媒工作室”，发行第九张个人专辑《太歌》。截至2020年，胡彦斌已发行十二张正式专辑，累计创作数百首歌曲。\n演艺事业外，胡彦斌还涉足音乐教育事业。2015年成立在线音乐教育平台“牛班”，2016年牛班音乐学校上海校区正式成立，之后在北京、深圳、广州、南京、成都等地陆续开设学习中心。2018年任南京艺术学院客座教授。\n胡彦斌热心社会活动。2004年为亚洲杯创作并演唱中英文版主题歌《宣言》。2006年为上海国际田径黄金大奖赛创作主题曲《And Want You Know》。2010年创作献唱的歌曲《我们一直都在》当选为2010上海世博会城市文明志愿者主题曲。2011年创作公益歌曲《孩子》，呼吁社会各界人士关注儿童；同年，被授予救助流浪儿童“手拉手——中国音乐志愿者在行动”形象大使。', '红颜、月光、诀别诗、你要的全拿走、男人KTV、劫后余生、不曾后悔、告诉我、你记得吗、昨天是你生日、一万光年、Waiting For You、包袱、空位、一年前、目不转睛、巴黎铁塔、不是不想、你这么晚了还没睡、有梦好甜蜜', '18岁全亚洲同步发片《文武双全》', '专辑《文武双全》突破四白金销量', '创作演唱04亚洲杯主题曲《宣言》', '22岁问鼎“内地最佳男歌手”奖项', '内地第一人在台湾举办售票演唱会', '牛班音乐学校创始人', '1999年参加“上海亚洲音乐节新人歌手大赛”在中国赛区获得银奖，总决赛获得“最高最具潜质大奖”，受到小室哲哉的赞赏，与步升音乐签下了合约。', '2000年以17岁的年龄推出个人首支单曲《玛丽莲·梦露》，并配唱中国动漫电视剧《我为歌狂》相关

{'歌手名': '胡期皓', '音乐链接': 'https://music.163.com/artist?id=31003906', '歌手信息': ['音乐万岁\nLong live my dear music']}
{'歌手名': '韩瞳', '音乐链接': 'https://music.163.com/artist?id=13007752', '歌手信息': ['韩瞳\n出生于2000年7月9日，7岁开始学习小提琴后学习钢琴 吉他 架子鼓及贝斯，十四岁留学美国，十八岁回国创办独立音乐工作室FUNIHILAND.后更名为費奈爾蘭德。韩瞳十七岁便开始学习音乐制作，回国后跟随许多国内著名制作人深度学习，后于2019年考上美国伯克利音乐学院。\n“有古典音乐的底子使他做出来的东西别具一格”\n在韩瞳身上能听到许多不同音乐风格的影子，大量音乐风格的学习积累与沉淀，影响着韩瞳。特别的旋律及巧妙的和声层使韩瞳的作品一直都有他的”独到之处"']}
{'歌手名': '胡夏', '音乐链接': 'https://music.163.com/artist?id=3273', '歌手信息': ['内地实力青年男歌手。在台湾选秀节目《超级星光大道》第六届夺下总冠军，以清澈温暖的深情演唱风格著称，并成为该节目第一个夺冠的内地歌手。因其清澈、温暖、深情的歌声被媒体誉为“清泉音”。个人首张专辑《胡 爱夏》在2010年12月17日在台湾正式发行。内地版于2011年1月10日在内地发行上市。2011年，为电影《那些年，我们一起追的女孩》献唱主题曲《那些年》，在港台各大权威排行榜占据多期冠军，MV在国外视频网站Youtube点击量超过3000万，目前仍在不断创造新的历史纪录。曾受邀担任「American Idol 美国偶像」总冠军Kelly Clarkson《无可救药 All I Ever Wanted》世界巡回演唱会台湾场的暖场嘉宾。2011年获第17届新加坡金曲奖“最佳新人奖”，成为有史以来最年轻、也是第一位获此奖项的内地男歌手。2012年凭单曲《那些年》获2011年度香港叱咤乐坛流行榜颁奖典礼“专业推荐叱咤十大”第二位及“叱咤乐坛我最喜爱的歌曲大奖”两项大奖，成为史上获得“叱咤乐坛我最喜爱的歌曲大奖”最年轻的歌手。代表作品：《那些年》，《爱夏》，《燃点》。\n', '2010年出道至今已发行

{'歌手名': '坤木Joymo', '音乐链接': 'https://music.163.com/artist?id=13790589', '歌手信息': ['致力于做好的音乐，除了音乐人不喜欢任何其他标签，除了大自然生态圈不属于任何圈子。']}
{'歌手名': 'KEY.L刘聪', '音乐链接': 'https://music.163.com/artist?id=12788674', '歌手信息': ['Key to Love\n混迹长沙的创作歌手。\nSUP_MUSIC创始人之一。\nC-BLOCK成员之一。 在Trap横行的年代，KEY.L选择把Trap的精髓抽离出来，而不是一味跟风。\n一路在C-BLOCK打磨，不管是自成一派的江湖style还是道系style，KEY.L一直做着街头的音乐。 2019.1.11号发行首张个人专辑「KEY to L」\n这把钥匙已经开启新的旅程 【微博】：刘聪的KEY 2019.1.11 发行首张个人专辑 「KEY to L」\n2020.10.10 发行第二张个人专辑「街区故事」 KEY.L刘聪 如货币般流通\n2021年，获得硬地围炉夜·网易云音乐原创盛典“最受欢迎男音乐人奖”\n2021年8月12日，参演音乐节目《披荆斩棘的哥哥》']}
{'歌手名': '空匪', '音乐链接': 'https://music.163.com/artist?id=30000975', '歌手信息': ['音乐爱好者 2001.0725 现在还不行 但是在努力 任何事宜微博私信空匪kongfei']}
{'歌手名': '枯木逢春', '音乐链接': 'https://music.163.com/artist?id=30284835', '歌手信息': ['你说也没有感动到流泪的故事\n这一辈子一无是处没有大志\n那些无奈的生活和睡不着的夜晚\n那些深入骨髓冰冷的孤单']}
{'歌手名': 'Kafe.Hu', '音乐链接': 'https://music.163.com/artist?id=12024274', '歌手信息': ['Kafe.Hu——摩登天空MDSK旗下Jazz Hiphop说唱歌手，嗓音沙哑而带有磁性，歌词睿智而带有黑色幽默，音乐中透着浓厚爵士的气息。\nKafe.Hu做真正的Jazz Hiphop，用他的语言表达着一个

KeyboardInterrupt: 